<a href="https://colab.research.google.com/github/nabiya18/MaintenacePreventive_DL/blob/main/Maintenancepredictive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Visualisation des éléments dans notre dataset

In [3]:
import pandas as pd

# Charger le dataset
data = pd.read_csv('dataset_pannes.csv')  # Remplacez ce chemin par l'emplacement réel de votre fichier

# 1. Afficher les premières lignes du dataset
print("Aperçu des premières lignes du dataset :")
print(data.head())  # Affiche les 5 premières lignes par défaut

# 2. Obtenir des informations générales sur le dataset (types de données, valeurs manquantes, etc.)
print("\nInformations générales sur les données :")
print(data.info())  # Affiche les informations sur les colonnes, types et valeurs manquantes

# 3. Statistiques descriptives du dataset pour mieux comprendre les données numériques
print("\nStatistiques descriptives du dataset :")
print(data.describe())  # Affiche des statistiques comme la moyenne, écart-type, min, max pour les colonnes numériques

# 4. Vérifier s'il y a des valeurs manquantes
print("\nVérifier les valeurs manquantes dans le dataset :")
print(data.isnull().sum())  # Compte le nombre de valeurs manquantes par colonne


Aperçu des premières lignes du dataset :
                  date  equipement_id  time  temperature  vibration   courant  \
0  2024-01-01 00:00:00              0     0    52.483571   6.399355  8.649643   
1  2024-01-01 00:01:00              0     1    49.318678   5.925634  9.715963   
2  2024-01-01 00:02:00              0     2    53.258443   5.061630  8.425160   
3  2024-01-01 00:03:00              0     3    57.645149   4.356063  9.399077   
4  2024-01-01 00:04:00              0     4    48.869233   5.702223  6.232771   

   alerts  panne  energy_consumption  cpu_usage  memory_usage  
0       0      0          400.971402  76.538337     60.354634  
1       0      0          447.250718  37.634755     53.877272  
2       0      0          470.648580  66.429584     39.491884  
3       1      0          507.483446  14.048207     94.795319  
4       0      0          551.208116  47.121475     83.626818  

Informations générales sur les données :
<class 'pandas.core.frame.DataFrame'>
RangeInd

Prétraitement des éléments de notre dataset

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

# Charger les données
df = pd.read_csv("dataset_pannes.csv", parse_dates=["date"])
df.sort_values(by=["equipement_id", "date"], inplace=True)

# Sélection des caractéristiques et normalisation
features = ["temperature", "vibration", "courant", "energy_consumption", "cpu_usage", "memory_usage"]
scaler = MinMaxScaler()
df[features] = scaler.fit_transform(df[features])

# Création de séquences temporelles
sequence_length = 20  # Longueur des séquences
X, y = [], []

def create_sequences(data, labels, seq_length):
    for i in range(len(data) - seq_length):
        X.append(data[i : i + seq_length])
        y.append(labels[i + seq_length])

for eq_id in df["equipement_id"].unique():
    eq_data = df[df["equipement_id"] == eq_id]
    create_sequences(eq_data[features].values, eq_data["panne"].values, sequence_length)

X, y = np.array(X), np.array(y)

# Sauvegarde des données prétraitées
np.save("X_sequences.npy", X)
np.save("y_labels.npy", y)


Création de notre modèle de Deep learning

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Charger les données
X = np.load("X_sequences.npy")
y = np.load("y_labels.npy")


# Définition du modèle amélioré
model = Sequential([
    LSTM(100, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
    Dropout(0.3),
    LSTM(100, return_sequences=True),
    Dropout(0.3),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25, activation='relu'),  # Couche supplémentaire
    Dense(10, activation='relu'),  # Ajout d'une nouvelle couche Dense
    Dense(1, activation='sigmoid')  # Sortie binaire
])

# Compilation du modèle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Sauvegarde du modèle avant entraînement
model.save("modele_lstm.keras")



/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Entraînement de notre modèle

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping

# Activer l'exécution dynamique
tf.config.run_functions_eagerly(True)

# Charger les données
X = np.load("X_sequences.npy")
y = np.load("y_labels.npy")

# Charger le modèle
model = load_model("modele_lstm.keras")

# Définir un callback pour l'arrêt anticipé
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Compilation du modèle (nécessaire après le chargement)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entraînement du modèle
history = model.fit(
    X, y, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping]
)

# Sauvegarde du modèle entraîné
model.save("modele_lstm_trained.keras")



Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 32 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.11/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


245/245 ━━━━━━━━━━━━━━━━━━━━ 149s 607ms/step - accuracy: 0.7043 - loss: 0.5941 - val_accuracy: 0.6990 - val_loss: 0.5379
Epoch 2/50
245/245 ━━━━━━━━━━━━━━━━━━━━ 193s 571ms/step - accuracy: 0.7339 - loss: 0.5225 - val_accuracy: 0.7357 - val_loss: 0.5448
Epoch 3/50
245/245 ━━━━━━━━━━━━━━━━━━━━ 143s 574ms/step - accuracy: 0.7326 - loss: 0.5255 - val_accuracy: 0.7378 - val_loss: 0.5109
Epoch 4/50
245/245 ━━━━━━━━━━━━━━━━━━━━ 144s 581ms/step - accuracy: 0.7294 - loss: 0.5153 - val_accuracy: 0.7250 - val_loss: 0.5271
Epoch 5/50
245/245 ━━━━━━━━━━━━━━━━━━━━ 203s 584ms/step - accuracy: 0.7426 - loss: 0.5160 - val_accuracy: 0.7423 - val_loss: 0.5346
Epoch 6/50
245/245 ━━━━━━━━━━━━━━━━━━━━ 143s 582ms/step - accuracy: 0.7270 - loss: 0.5234 - val_accuracy: 0.7388 - val_loss: 0.5009
Epoch 7/50
245/245 ━━━━━━━━━━━━━━━━━━━━ 147s 599ms/step - accuracy: 0.7432 - loss: 0.5134 - val_accuracy: 0.7398 - val_loss: 0.5023
Epoch 8/50
245/245 ━━━━━━━━━━━━━━━━━━━━ 141s 575ms/step - accuracy: 0.7361 - loss: 0.51

Evaluation de la performance du modèle

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# Charger les données
X = np.load("X_sequences.npy")
y = np.load("y_labels.npy")

# Charger le modèle entraîné
model = load_model("modele_lstm_trained.keras")

# Prédictions
predictions = model.predict(X)
predictions = (predictions > 0.45).astype(int)  # Convertir en 0 ou 1

# Évaluation du modèle
accuracy = accuracy_score(y, predictions)
conf_matrix = confusion_matrix(y, predictions)
class_report = classification_report(y, predictions)

# Affichage des résultats
print(f"Accuracy: {accuracy:.4f}")
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)

# Visualisation de la matrice de confusion
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["Pas de panne", "En panne"], yticklabels=["Pas de panne", "En panne"])
plt.xlabel("Prediction")
plt.ylabel("Actuel")
plt.title("Confusion Matrix")
plt.show()

# Visualisation des performances par classe
report_dict = classification_report(y, predictions, output_dict=True)
classes = list(report_dict.keys())[:-3]
f1_scores = [report_dict[cls]['f1-score'] for cls in classes]

plt.figure(figsize=(8, 5))
sns.barplot(x=classes, y=f1_scores, palette="viridis")
plt.xlabel("Classes")
plt.ylabel("F1-Score")
plt.title("Performance par classe")
plt.ylim(0, 1)
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'X_sequences.npy'